## COVID-19 Dashboard

### Introduction
* This code to retrieves COVID-19 data (updated daily) from Johns Hopkins University's [GitHub repo](https://github.com/CSSEGISandData) and prepares it for use in Tableau
* You can find my COVID-19 dashboard here: [COVID-19 Dashboard](https://public.tableau.com/profile/alfred.zou#!/vizhome/COVID-19_15852011851070/COVID-19)
* Please feel free to download and play around with it. It runs faster in Tableau public than online

### Main Features
* Timeline of COVID-19 confirmed cases
* Comparison of new confirmed cases and deaths between countries and states
* The number of active cases vs. available ICU beds for some countries

### Data Sources
* Data on coronavirus cases from Johns Hopkins University's [githup repo](https://github.com/CSSEGISandData)
* Population estimates from [worldometers](https://www.worldometers.info/world-population/population-by-country/)
* US ICU beds estimate from [AHA 2015 Annual Survey](https://www.sccm.org/Communications/Critical-Care-Statistics)
* Mexico ICU beds estimate from [apnews](https://apnews.com/ff99a460e304ffc67ab03d8a57b2e1ef)
* Australia and NZ ICU beds estimate from [ANZICS 2018 report](https://www.anzics.com.au/wp-content/uploads/2019/10/2018-ANZICS-CORE-Report.pdf)
* ICU beds estimate from 2012 academic paper ['The variability of critical care bed numbers in Europe'](https://link.springer.com/article/10.1007/s00134-012-2627-8/tables/2) by Rhodes et al. 
* ICU beds estimate from 2020 academic paper ['Critical Care Bed Capacity in Asian Countries and Regions'](https://www.researchgate.net/figure/Number-of-Critical-Care-Beds_tbl1_338520008) by Phua et al.
* % Populations over 65 per country (2019) from [worldbank](https://data.worldbank.org/indicator/SP.POP.65UP.TO.ZS)
* Number of Doctors and Nurses per 10,000 people per country from 2015 [WHO's World Health Stasticis report]( https://apps.who.int/iris/bitstream/handle/10665/170250/9789240694439_eng.pdf;jsessionid=FC99DC52822E45FBA833E47587CB567E?sequence=1)

### Code to extract and tranform data for Tableu

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# Increase maximum rows displayed in pandas DataFrame
pd.set_option('display.max_rows', 9999)
pd.set_option('display.max_columns', 50)

In [3]:
# data from John Hopkin's University
# The data is a time series of confirmed cases and deaths
confirmed_cases_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data" \
                      "/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
confirmed_deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data" \
             "/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
confirmed_recovered_url ="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data" \
            "/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

In [4]:
# Daily anity check if the erroneous data has been removed
foo = pd.read_csv(confirmed_cases_url)
foo[foo['Province/State']=='Recovered']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,...,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20


In [5]:
# Daily sanity check if the erroneous data has been removed
foo[foo['Province/State'].apply(str).str.contains('[Pp]rincess')]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,...,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20
37,Grand Princess,Canada,37.6489,-122.6655,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
231,Diamond Princess,Canada,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [6]:
# Remove erroneous Canada data
confirmed_cases = pd.read_csv(confirmed_cases_url)
confirmed_cases.drop(confirmed_cases[confirmed_cases['Province/State']=='Recovered'].index,inplace=True)
confirmed_cases.drop(confirmed_cases[confirmed_cases['Province/State']=='Diamond Princess'].index,inplace=True)

# Convert Province/State data to Country/Region data for Canada to match recoverd.csv 
foo = confirmed_cases.drop(columns=['Province/State','Lat','Long'])[confirmed_cases['Country/Region']=='Canada'].groupby('Country/Region').sum()
foo.insert(0,'Province/State',np.nan)
foo.insert(1,'Country/Region','Canada')
foo.insert(2,'Lat',pd.read_csv(confirmed_recovered_url).loc[pd.read_csv(confirmed_recovered_url)['Country/Region']=='Canada','Lat'].values)
foo.insert(3,'Long',pd.read_csv(confirmed_recovered_url).loc[pd.read_csv(confirmed_recovered_url)['Country/Region']=='Canada','Long'].values)
foo.reset_index(drop=True,inplace=True)
confirmed_cases.drop(confirmed_cases[confirmed_cases['Country/Region']=='Canada'].index,inplace=True)
confirmed_cases = pd.concat([confirmed_cases,foo])

# Melt into long format 
confirmed_cases = confirmed_cases.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Confirmed Cases')
confirmed_cases['Date']=pd.to_datetime(confirmed_cases['Date'],format='%m/%d/%y')
confirmed_cases.sort_values(['Country/Region','Province/State','Date'],inplace=True)
confirmed_cases.reset_index(drop=True,inplace=True)
confirmed_cases

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0
...,...,...,...,...,...,...
29849,NaN,Zimbabwe,-20.0,30.0,2020-05-14,37
29850,NaN,Zimbabwe,-20.0,30.0,2020-05-15,42
29851,NaN,Zimbabwe,-20.0,30.0,2020-05-16,42
29852,NaN,Zimbabwe,-20.0,30.0,2020-05-17,44


In [7]:
# Remove erroneous Canada data
confirmed_deaths = pd.read_csv(confirmed_deaths_url)
confirmed_deaths.drop(confirmed_deaths[confirmed_deaths['Province/State']=='Recovered'].index,inplace=True)
confirmed_deaths.drop(confirmed_deaths[confirmed_deaths['Province/State']=='Diamond Princess'].index,inplace=True)

# Convert Province/State data to Country/Region data for Canada to match recoverd.csv 
foo = confirmed_deaths.drop(columns=['Province/State','Lat','Long'])[confirmed_deaths['Country/Region']=='Canada'].groupby('Country/Region').sum()
foo.insert(0,'Province/State',np.nan)
foo.insert(1,'Country/Region','Canada')
foo.insert(2,'Lat',pd.read_csv(confirmed_recovered_url).loc[pd.read_csv(confirmed_recovered_url)['Country/Region']=='Canada','Lat'].values)
foo.insert(3,'Long',pd.read_csv(confirmed_recovered_url).loc[pd.read_csv(confirmed_recovered_url)['Country/Region']=='Canada','Long'].values)
foo.reset_index(drop=True,inplace=True)
confirmed_deaths.drop(confirmed_deaths[confirmed_deaths['Country/Region']=='Canada'].index,inplace=True)
confirmed_deaths = pd.concat([confirmed_deaths,foo])

# Melt into long format 
confirmed_deaths = confirmed_deaths.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Confirmed Deaths')
confirmed_deaths['Date']=pd.to_datetime(confirmed_deaths['Date'],format='%m/%d/%y')
confirmed_deaths.sort_values(['Country/Region','Province/State','Date'],inplace=True)
confirmed_deaths.reset_index(drop=True,inplace=True)
confirmed_deaths

,Province/State,Country/Region,Lat,Long,Date,Confirmed Deaths
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0
...,...,...,...,...,...,...
29849,NaN,Zimbabwe,-20.0,30.0,2020-05-14,4
29850,NaN,Zimbabwe,-20.0,30.0,2020-05-15,4
29851,NaN,Zimbabwe,-20.0,30.0,2020-05-16,4
29852,NaN,Zimbabwe,-20.0,30.0,2020-05-17,4


In [8]:
# Load data and remove wrong data
confirmed_recovered = pd.read_csv(confirmed_recovered_url)

# Fix up some mis matched cordinates with confirmed cases and deaths csv
foo = ['Mozambique','Syria', 'Timor-Leste']
bar = pd.read_csv(confirmed_deaths_url)
for country in foo:
    mask = confirmed_recovered['Country/Region']==country
    mask2 = bar['Country/Region']==country
    confirmed_recovered.loc[mask,'Lat']=bar.loc[mask2,'Lat'].values
    confirmed_recovered.loc[mask,'Long']=bar.loc[mask2,'Long'].values

# Melt into long form
confirmed_recovered = confirmed_recovered.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Confirmed Recovered')
confirmed_recovered['Date']=pd.to_datetime(confirmed_recovered['Date'],format='%m/%d/%y')
confirmed_recovered.sort_values(['Country/Region','Province/State','Date'],inplace=True)
confirmed_recovered.reset_index(drop=True,inplace=True)
confirmed_recovered

,Province/State,Country/Region,Lat,Long,Date,Confirmed Recovered
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0
...,...,...,...,...,...,...
29849,NaN,Zimbabwe,-20.0,30.0,2020-05-14,13
29850,NaN,Zimbabwe,-20.0,30.0,2020-05-15,13
29851,NaN,Zimbabwe,-20.0,30.0,2020-05-16,13
29852,NaN,Zimbabwe,-20.0,30.0,2020-05-17,17


In [9]:
# Join the data together
confirmed_cases_deaths = confirmed_cases.merge(confirmed_deaths)
confirmed_cases_deaths = confirmed_cases_deaths.merge(confirmed_recovered,how='outer')
confirmed_cases_deaths

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases,Confirmed Deaths,Confirmed Recovered
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0,0,0
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0,0,0
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0,0,0
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0,0,0
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0,0,0
...,...,...,...,...,...,...,...,...
29849,NaN,Zimbabwe,-20.0,30.0,2020-05-14,37,4,13
29850,NaN,Zimbabwe,-20.0,30.0,2020-05-15,42,4,13
29851,NaN,Zimbabwe,-20.0,30.0,2020-05-16,42,4,13
29852,NaN,Zimbabwe,-20.0,30.0,2020-05-17,44,4,17


In [10]:
# Check for any nulls from outer joins
mask = ['Date','Lat','Long','Province/State','Country/Region','Confirmed Cases','Confirmed Deaths','Confirmed Recovered']
mask2 = ['Confirmed Recovered','Confirmed Cases']
confirmed_cases_deaths.loc[confirmed_cases_deaths[mask2].isnull().any(axis=1),mask]

,Date,Lat,Long,Province/State,Country/Region,Confirmed Cases,Confirmed Deaths,Confirmed Recovered


In [11]:
# Create arrays of unique country_region and province_state pairs to iterate through
countries_regions = confirmed_cases_deaths['Country/Region'].unique() 
provinces_states = confirmed_cases_deaths['Province/State'].unique()

# Create new columns
# Because countries will outbreak at different times, it is recommended to compare them on a common time scale; such as the days since 100 confirmed cases for each country
# Hence the new column Days_Since_100. A day of 1 means 1 day after reaching 100 confirmed cases
# New_Cases and New_Deaths column is used to determine the daily number of new confirmed cases and deaths
confirmed_cases_deaths['Days Since 100 Cases'] = 0
confirmed_cases_deaths['New Cases'] = 0
confirmed_cases_deaths['New Deaths'] = 0

# Iterating through each country/region, create a filtering mask
for cr in countries_regions:
    mask1 = confirmed_cases_deaths['Country/Region']== cr
    
    # Iterating through each province/state, create a filtering mask
    for ps in provinces_states:
        # This lets us filter for provinces/states that have a null value
        if type(ps)==float:
            mask2 = confirmed_cases_deaths['Province/State'].isnull()
        else:
            mask2 = confirmed_cases_deaths['Province/State']== ps
        
        # Filtering by applying country/region and province/state mask, find the difference between the Confirmed cases to get the new daily cases and deaths
        confirmed_cases_deaths.loc[mask1 & mask2,'New Cases'] =confirmed_cases_deaths.loc[mask1 & mask2,'Confirmed Cases'].diff()
        confirmed_cases_deaths.loc[mask1 & mask2,'New Deaths'] =confirmed_cases_deaths.loc[mask1 & mask2,'Confirmed Deaths'].diff()
        
        # Initialise empty count values
        # These will reset with each province/state
        count = 0     
        
        # For each row in the filtered df, check if the Confirmed cases have passed each threshold
        # If so, apply an accumalating count
        # If not, assign np.nan
        for row in confirmed_cases_deaths.loc[mask1 & mask2,'Confirmed Cases'].index:                        
            if confirmed_cases_deaths.loc[row,'Confirmed Cases'] > 100:
                confirmed_cases_deaths.loc[row,'Days Since 100 Cases'] = count
                count += 1
            else:
                confirmed_cases_deaths.loc[row,'Days Since 100 Cases'] = np.nan
            
# Because the first row in new_cases is always going to be null using .diff(), assign new_cases = confirmed_cases_deaths for first row (or null) values
nan_mask = confirmed_cases_deaths['New Cases'].isnull()
confirmed_cases_deaths.loc[nan_mask,'New Cases'] = confirmed_cases_deaths['Confirmed Cases']
nan_mask = confirmed_cases_deaths['New Deaths'].isnull()
confirmed_cases_deaths.loc[nan_mask,'New Deaths'] = confirmed_cases_deaths['Confirmed Deaths']
# Convert floats into ints
confirmed_cases_deaths['New Cases']=pd.to_numeric(confirmed_cases_deaths['New Cases'],downcast='signed')

In [12]:
# Sanity check the Days Since X Cases
mask1 = confirmed_cases_deaths['Country/Region']== 'China'
mask2 = confirmed_cases_deaths['Province/State']== confirmed_cases_deaths.loc[mask1,'Province/State'].unique()[-1]
confirmed_cases_deaths.loc[mask1 & mask2]

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases,Confirmed Deaths,Confirmed Recovered,Days Since 100 Cases,New Cases,New Deaths
8850,Zhejiang,China,29.1832,120.0934,2020-01-22,10,0,0,NaN,10,0.0
8851,Zhejiang,China,29.1832,120.0934,2020-01-23,27,0,0,NaN,17,0.0
8852,Zhejiang,China,29.1832,120.0934,2020-01-24,43,0,1,NaN,16,0.0
8853,Zhejiang,China,29.1832,120.0934,2020-01-25,62,0,1,NaN,19,0.0
8854,Zhejiang,China,29.1832,120.0934,2020-01-26,104,0,1,0.0,42,0.0
8855,Zhejiang,China,29.1832,120.0934,2020-01-27,128,0,1,1.0,24,0.0
8856,Zhejiang,China,29.1832,120.0934,2020-01-28,173,0,3,2.0,45,0.0
8857,Zhejiang,China,29.1832,120.0934,2020-01-29,296,0,3,3.0,123,0.0
8858,Zhejiang,China,29.1832,120.0934,2020-01-30,428,0,4,4.0,132,0.0
8859,Zhejiang,China,29.1832,120.0934,2020-01-31,538,0,14,5.0,110,0.0


In [13]:
# Sanity check the Days Since X Cases
mask1 = confirmed_cases_deaths['Country/Region']== 'India'
confirmed_cases_deaths.loc[mask1]

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases,Confirmed Deaths,Confirmed Recovered,Days Since 100 Cases,New Cases,New Deaths
15340,NaN,India,21.0,78.0,2020-01-22,0,0,0,NaN,0,0.0
15341,NaN,India,21.0,78.0,2020-01-23,0,0,0,NaN,0,0.0
15342,NaN,India,21.0,78.0,2020-01-24,0,0,0,NaN,0,0.0
15343,NaN,India,21.0,78.0,2020-01-25,0,0,0,NaN,0,0.0
15344,NaN,India,21.0,78.0,2020-01-26,0,0,0,NaN,0,0.0
15345,NaN,India,21.0,78.0,2020-01-27,0,0,0,NaN,0,0.0
15346,NaN,India,21.0,78.0,2020-01-28,0,0,0,NaN,0,0.0
15347,NaN,India,21.0,78.0,2020-01-29,0,0,0,NaN,0,0.0
15348,NaN,India,21.0,78.0,2020-01-30,1,0,0,NaN,1,0.0
15349,NaN,India,21.0,78.0,2020-01-31,1,0,0,NaN,0,0.0


In [14]:
# Load in data which maps countries to continents
continent_countries = pd.read_excel('Countries.xlsx',usecols = ['Continent','Country','ICU Beds','Doctors per 10000','Nurses per 10000','ICU Beds per 100000'])

In [15]:
# Map the continents to the countries
confirmed_cases_deaths = confirmed_cases_deaths.merge(continent_countries,how='left',left_on='Country/Region',right_on='Country')
confirmed_cases_deaths.drop(columns='Country',inplace=True)
confirmed_cases_deaths

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases,Confirmed Deaths,Confirmed Recovered,Days Since 100 Cases,New Cases,New Deaths,Continent,ICU Beds,Doctors per 10000,Nurses per 10000,ICU Beds per 100000
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0,0,0,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0,0,0,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0,0,0,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0,0,0,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0,0,0,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29849,NaN,Zimbabwe,-20.0,30.0,2020-05-14,37,4,13,NaN,0,0.0,Africa,NaN,0.8,13.4,NaN
29850,NaN,Zimbabwe,-20.0,30.0,2020-05-15,42,4,13,NaN,5,0.0,Africa,NaN,0.8,13.4,NaN
29851,NaN,Zimbabwe,-20.0,30.0,2020-05-16,42,4,13,NaN,0,0.0,Africa,NaN,0.8,13.4,NaN
29852,NaN,Zimbabwe,-20.0,30.0,2020-05-17,44,4,17,NaN,2,0.0,Africa,NaN,0.8,13.4,NaN


In [16]:
pop_above_65 = pd.read_csv('Pop Above 65.csv')
pop_above_65.columns = ['Country/Region','% Population Above 65']
confirmed_cases_deaths = confirmed_cases_deaths.merge(pop_above_65,how='left')

In [17]:
pop_above_65.sort_values('% Population Above 65',ascending=False)

,Country/Region,% Population Above 65
117,Japan,27.576370
114,Italy,22.751680
192,Portugal,21.953858
73,Finland,21.720788
87,Greece,21.655272
53,Germany,21.461962
19,Bulgaria,21.021914
66,Euro area,20.600288
97,Croatia,20.445433
157,Malta,20.349324


In [18]:
def create_ref_line(starting_value):
    '''
    Creates two reference points for Tableau to draw its trendline for a certain starting_value of confirmed cases
    '''
    length = 130
    x = list(range(length))
    y = np.zeros(length)
    df = pd.DataFrame({f'Days Since {starting_value} Cases':x,'Doubles in 2 Days':y,'Doubles in 3 Days':y,'Doubles in 4 Days':y,'Doubles in 5 Days':y,
                        'Doubles in 6 Days':y,'Doubles in 7 Days':y,'Doubles in 8 Days':y,'Doubles in 9 Days':y,'Doubles in 10 Days':y})
    df.iloc[0,1:] = starting_value
    
    for row in range(1,length):
        for step in range(2,11):
            df.loc[row,f'Doubles in {step} Days'] = df.loc[row-1,f'Doubles in {step} Days']*pow(2,1/step)
    return df

In [19]:
reference_lines = create_ref_line(100)
reference_lines = reference_lines.melt(id_vars=['Days Since 100 Cases']
                                       ,var_name='Doubles in N Days',value_name='Reference Cases')
reference_lines

,Days Since 100 Cases,Doubles in N Days,Reference Cases
0,0,Doubles in 2 Days,1.000000e+02
1,1,Doubles in 2 Days,1.414214e+02
2,2,Doubles in 2 Days,2.000000e+02
3,3,Doubles in 2 Days,2.828427e+02
4,4,Doubles in 2 Days,4.000000e+02
5,5,Doubles in 2 Days,5.656854e+02
6,6,Doubles in 2 Days,8.000000e+02
7,7,Doubles in 2 Days,1.131371e+03
8,8,Doubles in 2 Days,1.600000e+03
9,9,Doubles in 2 Days,2.262742e+03


In [20]:
# Create folder and save csv's for Tableau to read
!mkdir Data
confirmed_cases_deaths.to_csv('Data/confirmed_cases_deaths.csv')
reference_lines.to_csv('Data/reference_lines.csv')

A subdirectory or file Data already exists.
